In [10]:
import pandas as pd
import json
import random

In [11]:
import arff
import pandas as pd

with open("nfr.arff", "r") as f:
    arff_data = arff.load(f)
data = arff_data['data']
columns = [attr[0] for attr in arff_data['attributes']]

df = pd.DataFrame(data, columns=columns)

In [12]:
df['class'] = df['class'].apply(lambda x: "FR" if x == "F" else "NFR")
df.head(10)

,ProjectID,RequirementText,class
0,1,The system shall refresh the display every 60 ...,NFR
1,1,The application shall match the color of the s...,NFR
2,1,If projected the data must be readable. On ...,NFR
3,1,The product shall be available during normal ...,NFR
4,1,If projected the data must be understandable...,NFR
5,1,The product shall ensure that it can only be a...,NFR
6,1,The product shall be intuitive and self-explan...,NFR
7,1,The product shall respond fast to keep up-to-d...,NFR
8,1,The system shall have a MDI form that allows f...,FR
9,1,The system shall display Events in a vertical ...,FR


In [13]:
jsonl_data = []
for index, row in df.iterrows():
    jsonl_data.append({
        "messages": [
            {"role": "user", "content": row["RequirementText"]},
            {"role": "assistant", "content": row["class"]}
        ]
    })

In [14]:
with open("data.jsonl", "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")
file_path = "data.jsonl"
with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]

random.shuffle(data)

split_index = int(0.8 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

with open("train_data.jsonl", "w") as train_file:
    for entry in train_data:
        train_file.write(json.dumps(entry) + "\n")

with open("test_data.jsonl", "w") as test_file:
    for entry in test_data:
        test_file.write(json.dumps(entry) + "\n")

In [1]:
from openai import OpenAI

with open("../openai_api_key.txt", "r") as file:
        openai_api_key = file.read().strip()
        client = OpenAI(api_key=openai_api_key)
file_response = client.files.create(
    file=open("train_data.jsonl", "rb"), purpose="fine-tune"
)
file_obj_id = file_response.id
print(f"Uploaded file! Object id: {file_obj_id}")

Uploaded file! Object id: file-2BT6pgO1lzSFn0uj1u4Cb0Dw


In [16]:
fine_tune = client.fine_tuning.jobs.create(training_file=file_obj_id, model="gpt-4o-mini-2024-07-18")
fine_tune_id = fine_tune.id
print(f"Fine_tune ID: {fine_tune_id}")

Fine_tune ID: ftjob-h46WGk5Xixhxuid4FOqy4D1B


In [17]:
client.fine_tuning.jobs.retrieve(fine_tune_id)

FineTuningJob(id='ftjob-h46WGk5Xixhxuid4FOqy4D1B', created_at=1730672400, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-dOwXPJQ9qE4xluNhMVuFywBO', result_files=[], seed=564078495, status='validating_files', trained_tokens=None, training_file='file-5G0dZhyG8KDrveRh3el2jnLb', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [18]:
with open("test_data.jsonl", "rb") as f:
    test_data = [json.loads(line) for line in f]
    
actual_labels = []
predicted_labels = []

for entry in test_data:
    message = entry["messages"][0]["content"]
    label = entry["messages"][1]["content"]

    actual_labels.append(label)

    completion = client.chat.completions.create(
        model=f"ft:gpt-4o-mini-2024-07-18:personal::APdkJsuS",
        messages=[{"role": "user", "content": message}]
    )

    predicted_label = completion.choices[0].message.content.strip()
    predicted_labels.append(predicted_label)

In [19]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels, average="weighted")
f1 = f1_score(actual_labels, predicted_labels, average="weighted")
recall = recall_score(actual_labels, predicted_labels, average="weighted")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.944
Precision: 0.9444755090809026
Recall: 0.944
F1 Score: 0.9436577249575552
